In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install pillow

In [3]:
import zipfile
import os

zip_file_path = "/content/drive/MyDrive/MRI/MRI.zip"

extract_to_directory = "/content/MRI"
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to_directory)

In [4]:
from PIL import Image
import os

def resize_images_in_folder(folder_path, target_size):
    for root, dirs, files in os.walk(folder_path):
        for filename in files:
            filepath = os.path.join(root, filename)
            try:
                if is_image_file(filepath):
                    with Image.open(filepath) as img:
                        if img.mode != 'RGB':
                            img = img.convert('RGB')

                        img_resized = img.resize(target_size, Image.LANCZOS)
                        img_resized.save(filepath)
                else:
                    print(f"{filename} is not a recognized image file. Skipping...")
            except Exception as e:
                print(f"Error processing {filename}: {e}")

def is_image_file(filepath):
    try:
        Image.open(filepath).verify()
        return True
    except:
        return False

train_folder = "/content/MRI/Training"
test_folder = "/content/MRI/Testing"

width = 256
height = 256

target_size = (width, height)

resize_images_in_folder(train_folder, target_size)

resize_images_in_folder(test_folder, target_size)

In [5]:
!pip install tensorflow keras

In [7]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    '/content/MRI/Training',
    target_size=(256, 256),
    batch_size=32,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    '/content/MRI/Testing',
    target_size=(256, 256),
    batch_size=32,
    class_mode='categorical')

model = Sequential([
    Flatten(input_shape=(256, 256, 3)),
    Dense(512, activation='relu'),
    Dense(256, activation='relu'),
    Dense(128, activation='relu'),
    Dense(4, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples ,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples
)


Found 5712 images belonging to 4 classes.
Found 1311 images belonging to 4 classes.
Epoch 1/10
178/178 [==============================] - 18s 89ms/step - loss: 3.7437 - accuracy: 0.5583 - val_loss: 0.8232 - val_accuracy: 0.7281
Epoch 2/10
178/178 [==============================] - 17s 94ms/step - loss: 0.7015 - accuracy: 0.7493 - val_loss: 0.6179 - val_accuracy: 0.7805
Epoch 3/10
178/178 [==============================] - 13s 75ms/step - loss: 0.5709 - accuracy: 0.7887 - val_loss: 0.5303 - val_accuracy: 0.8109
Epoch 4/10
178/178 [==============================] - 14s 81ms/step - loss: 0.4092 - accuracy: 0.8495 - val_loss: 0.5416 - val_accuracy: 0.7953
Epoch 5/10
178/178 [==============================] - 13s 74ms/step - loss: 0.3934 - accuracy: 0.8518 - val_loss: 0.4497 - val_accuracy: 0.8336
Epoch 6/10
178/178 [==============================] - 14s 79ms/step - loss: 0.3832 - accuracy: 0.8546 - val_loss: 0.6281 - val_accuracy: 0.8016
Epoch 7/10
178/178 [==============================] 

In [8]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 196608)            0         
                                                                 
 dense_2 (Dense)             (None, 512)               100663808 
                                                                 
 dense_3 (Dense)             (None, 256)               131328    
                                                                 
 dense_4 (Dense)             (None, 128)               32896     
                                                                 
 dense_5 (Dense)             (None, 4)                 516       
                                                                 
Total params: 100828548 (384.63 MB)
Trainable params: 100828548 (384.63 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
